<a href="https://colab.research.google.com/github/Akazelll/RiceLeafDisease/blob/main/final_efficientnetB0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
loki4514_rice_leaf_diseases_detection_path = kagglehub.dataset_download('loki4514/rice-leaf-diseases-detection')

print('Data source import complete.')


Using Colab cache for faster access to the 'rice-leaf-diseases-detection' dataset.
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Output streaming akan dipotong hingga 5000 baris terakhir.
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/Narrow Brown Leaf Spot/aug_0_511.jpg
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/Narrow Brown Leaf Spot/aug_0_2482.jpg
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/Narrow Brown Leaf Spot/aug_0_9857.jpg
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/Narrow Brown Leaf Spot/aug_0_2720.jpg
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/Narrow Brown Leaf Spot/aug_0_41.jpg
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/Narrow Brown Leaf Spot/aug_0_8477.jpg
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/Narrow Brown Leaf Spot/aug_0_3440.jpg
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/Narrow Brown Leaf Spot/aug_0_8279.jpg
/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_AUG/Rice_Leaf_AUG/N

In [ ]:
# ==============================================================================
# 1. DATA COLLECTING
# ==============================================================================
import os
import random
import numpy as np
import tensorflow as tf

print("\n===== 1. DATA COLLECTING ====")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Corrected SOURCE_DIR to use the path from kagglehub.dataset_download
# The variable 'loki4514_rice_leaf_diseases_detection_path' from the first cell holds the base path.
SOURCE_DIR = os.path.join(loki4514_rice_leaf_diseases_detection_path, 'Rice_Leaf_AUG', 'Rice_Leaf_AUG')

total_images = 0
class_names = []

for cls in sorted(os.listdir(SOURCE_DIR)):
    cls_path = os.path.join(SOURCE_DIR, cls)
    if os.path.isdir(cls_path):
        count = len(os.listdir(cls_path))
        class_names.append(cls)
        total_images += count
        print(f"Class '{cls}': {count} images")

print(f"\nTotal images collected : {total_images}")
print(f"Total classes         : {len(class_names)}")

In [ ]:
# ==============================================================================
# (VISUAL SAMPLE)
# ==============================================================================
import os
import matplotlib.pyplot as plt
from PIL import Image

print("\n===== Sampel Visual Gambar Asli =====")

sample_class = class_names[0]
sample_img = os.listdir(os.path.join(SOURCE_DIR, sample_class))[0]

img = Image.open(os.path.join(SOURCE_DIR, sample_class, sample_img))

plt.figure(figsize=(4,4))
plt.imshow(img)
plt.title(f"Contoh Citra Asli - Kelas {sample_class}")
plt.axis('off')
plt.show()


In [7]:
# ==============================================================================
# 3. DATA SPLITTING (80% TRAIN | 20% TEST)
# ==============================================================================
import os
import shutil
from sklearn.model_selection import train_test_split

print("\n===== 3. DATA SPLITTING =====")

TRAIN_DIR = '/kaggle/working/train'
TEST_DIR  = '/kaggle/working/test'

# Bersihkan folder lama agar konsisten
for d in [TRAIN_DIR, TEST_DIR]:
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d)

# Split data per kelas (stratified by folder)
for cls in class_names:
    cls_path = os.path.join(SOURCE_DIR, cls)
    files = os.listdir(cls_path)

    train_f, test_f = train_test_split(
        files,
        test_size=0.2,
        random_state=SEED,
        shuffle=True
    )

    os.makedirs(os.path.join(TRAIN_DIR, cls))
    os.makedirs(os.path.join(TEST_DIR, cls))

    for f in train_f:
        shutil.copy(
            os.path.join(cls_path, f),
            os.path.join(TRAIN_DIR, cls, f)
        )
    for f in test_f:
        shutil.copy(
            os.path.join(cls_path, f),
            os.path.join(TEST_DIR, cls, f)
        )

print("Data split completed successfully (80% train | 20% test).")



===== 3. DATA SPLITTING =====
Data split completed successfully (80% train | 20% test).


In [ ]:
# ==============================================================================
# 4. DATA BALANCING ANALYSIS
# ==============================================================================
print("\n===== 4. DATA BALANCING =====")

counts = []
for cls in class_names:
    c = len(os.listdir(os.path.join(TRAIN_DIR, cls)))
    counts.append(c)
    print(f"{cls}: {c} images")

plt.figure(figsize=(10,5))
plt.bar(class_names, counts)
plt.xticks(rotation=45, ha='right')
plt.ylabel("Jumlah Citra")
plt.title("Distribusi Data Training per Kelas")
plt.tight_layout()
plt.show()

print("Dataset relatif seimbang → tidak dilakukan resampling.")


In [ ]:
# ==============================================================================
# 5. DATA GENERATOR
# ==============================================================================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

print("\n===== 5. DATA GENERATOR =====")

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_ds = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

test_ds = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"Training samples : {train_ds.samples}")
print(f"Testing samples  : {test_ds.samples}")
print(f"Classes detected: {len(train_ds.class_indices)}")


In [ ]:
# ==============================================================================
# VISUALISASI
# SEBELUM vs SESUDAH PREPROCESSING
# ==============================================================================
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

print("\n===== PERBANDINGAN CITRA SEBELUM & SESUDAH PREPROCESSING =====")

# ==========================
# 1. CITRA SEBELUM PREPROCESSING
# ==========================
sample_class = sorted(os.listdir(TRAIN_DIR))[0]
sample_img_name = os.listdir(os.path.join(TRAIN_DIR, sample_class))[0]
original_img_path = os.path.join(TRAIN_DIR, sample_class, sample_img_name)

original_img = Image.open(original_img_path)
original_array = np.array(original_img)

print("📌 INFORMASI CITRA SEBELUM PREPROCESSING")
print(f"- Resolusi         : {original_img.size} (width x height)")
print(f"- Rentang pixel    : {original_array.min()} – {original_array.max()}")
print(f"- Tipe data pixel  : {original_array.dtype}")

# ==========================
# 2. CITRA SETELAH PREPROCESSING
# ==========================
img_batch, _ = next(train_ds)
processed_img = img_batch[0]

print("\n📌 INFORMASI CITRA SETELAH PREPROCESSING")
print(f"- Resolusi         : {processed_img.shape[1]} x {processed_img.shape[0]}")
print(f"- Rentang pixel    : {processed_img.min():.2f} – {processed_img.max():.2f}")
print(f"- Tipe data pixel  : {processed_img.dtype}")

# Normalisasi hanya untuk VISUALISASI
processed_img_vis = (
    (processed_img - processed_img.min()) /
    (processed_img.max() - processed_img.min())
)

# ==========================
# 3. VISUALISASI PERBANDINGAN
# ==========================
plt.figure(figsize=(9,4))

# Sebelum
plt.subplot(1,2,1)
plt.imshow(original_img)
plt.title(
    f"Sebelum Preprocessing\n"
    f"Resolusi: {original_img.size}\n"
    f"Pixel: 0–255"
)
plt.axis('off')

# Sesudah
plt.subplot(1,2,2)
plt.imshow(processed_img_vis)
plt.title(
    f"Sesudah Preprocessing\n"
    f"Resolusi: {processed_img.shape[1]}×{processed_img.shape[0]}\n"
    f"Pixel: Ternormalisasi"
)
plt.axis('off')

plt.suptitle(
    "Perbandingan Citra Sebelum dan Sesudah Preprocessing",
    fontsize=12,
    fontweight='bold'
)

plt.tight_layout()
plt.show()


In [ ]:
# ==============================================================================
# 6. MODELLING
# ==============================================================================
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0

print("\n===== 6. MODELLING =====")

NUM_CLASSES = len(train_ds.class_indices)

base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze backbone (sesuai konfigurasi sebelumnya)
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
# ==============================================================================
# 7. TRAINING
# ==============================================================================
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

print("\n===== 7. TRAINING =====")

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=6,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=15,
    callbacks=[early_stop],
    verbose=2
)


In [ ]:
# ==============================================================================
# 8. MODEL EVALUATION (METRIC)
# ==============================================================================
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("\n===== 8. MODEL EVALUATION =====")

test_ds.reset()
pred_probs = model.predict(test_ds, verbose=0)
pred_labels = np.argmax(pred_probs, axis=1)
true_labels = test_ds.classes

acc = accuracy_score(true_labels, pred_labels)
prec = precision_score(true_labels, pred_labels, average='weighted')
rec = recall_score(true_labels, pred_labels, average='weighted')
f1 = f1_score(true_labels, pred_labels, average='weighted')

print(f"Accuracy  : {acc:.4f}")
print(f"Precision : {prec:.4f}")
print(f"Recall    : {rec:.4f}")
print(f"F1-score  : {f1:.4f}")


In [ ]:
# ==============================================================================
# 8. MODEL EVALUATION (ACCURACY & LOSS CURVE)
# ==============================================================================
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))

# -----------------------------
# Accuracy Curve
# -----------------------------
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curve')
plt.legend()
plt.grid(True)

# -----------------------------
# Loss Curve
# -----------------------------
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# ==============================================================================
# VISUALISASI AKURASI AKHIR & CONFUSION MATRIX
# ==============================================================================
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

print("\n===== VISUALISASI AKURASI & CONFUSION MATRIX =====")

# ==========================
# 1. BAR CHART AKURASI AKHIR
# ==========================
plt.figure(figsize=(5,3))
plt.bar(['Accuracy'], [acc])
plt.ylim(0,1)
plt.title("Final Test Accuracy")

plt.text(
    0, acc + 0.02,
    f"{acc*100:.2f}%",
    ha='center',
    fontweight='bold'
)

plt.show()

# ==========================
# 2. CONFUSION MATRIX
# ==========================
idx_to_class = {v:k for k,v in test_ds.class_indices.items()}
labels = [idx_to_class[i] for i in range(NUM_CLASSES)]

cm = confusion_matrix(true_labels, pred_labels)

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=labels
)

plt.figure(figsize=(10,8))
disp.plot(
    cmap=plt.cm.Blues,
    xticks_rotation=90,
    values_format='d'
)
plt.title("Confusion Matrix - Test Data")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("\n===== 9. SAMPLE PREDICTION CHECK (PER CLASS) =====")

# Mapping index → nama kelas
idx_to_class = {v: k for k, v in test_ds.class_indices.items()}
class_labels = [idx_to_class[i] for i in range(NUM_CLASSES)]

# --------------------------------------------------
# Ambil 1 sampel unik untuk setiap kelas
# --------------------------------------------------
samples = {}

test_ds.reset()
for imgs, lbls in test_ds:
    for img, lbl in zip(imgs, lbls):
        cls_idx = np.argmax(lbl)
        if cls_idx not in samples:
            samples[cls_idx] = img
        if len(samples) == NUM_CLASSES:
            break
    if len(samples) == NUM_CLASSES:
        break

# --------------------------------------------------
# Prediksi & Visualisasi
# --------------------------------------------------
plt.figure(figsize=(15, 15))

for i, cls_idx in enumerate(sorted(samples.keys())):
    img = samples[cls_idx]

    pred = model.predict(
        np.expand_dims(img, axis=0),
        verbose=0
    )[0]

    pred_idx = np.argmax(pred)
    confidence = np.max(pred) * 100

    true_label = class_labels[cls_idx]
    pred_label = class_labels[pred_idx]
    correct = "BENAR" if cls_idx == pred_idx else "SALAH"

    # Normalisasi hanya untuk visualisasi
    img_vis = (img - img.min()) / (img.max() - img.min())

    plt.subplot(3, 3, i + 1)
    plt.imshow(img_vis)
    plt.axis("off")
    plt.title(
        f"True      : {true_label}\n"
        f"Predict   : {pred_label}\n"
        f"Confidence: {confidence:.2f}%\n"
        f"Status    : {correct}",
        fontsize=11
    )

    # Output teks (untuk log / laporan)
    print(
        f"Class: {true_label} | "
        f"Predict: {pred_label} | "
        f"Confidence: {confidence:.2f}% | "
        f"Status: {correct}"
    )

plt.suptitle(
    "Sample Prediction Check – Satu Sampel per Kelas",
    fontsize=14,
    fontweight="bold"
)
plt.tight_layout()
plt.show()

In [ ]:
# ==============================================================================
# 10. MODEL HEALTH CHECK (COMPREHENSIVE)
# ==============================================================================

import numpy as np

print("\n===== 10. MODEL HEALTH CHECK =====")

# ------------------------------------------------------------------
# 1. OVERFITTING / UNDERFITTING CHECK
# ------------------------------------------------------------------
train_acc = history.history['accuracy'][-1]
val_acc   = history.history['val_accuracy'][-1]
acc_gap   = abs(train_acc - val_acc)

print("\n[1] Overfitting Check")
print(f"Train Accuracy      : {train_acc:.4f}")
print(f"Validation Accuracy : {val_acc:.4f}")
print(f"Accuracy Gap        : {acc_gap:.4f}")

if acc_gap <= 0.05:
    fit_status = "SEHAT (Tidak overfitting)"
elif train_acc > val_acc:
    fit_status = "OVERFITTING"
else:
    fit_status = "UNDERFITTING"

print(f"Status              : {fit_status}")

# ------------------------------------------------------------------
# 2. DATA IMBALANCE CHECK
# ------------------------------------------------------------------
class_counts = np.bincount(train_ds.classes)
imbalance_ratio = class_counts.max() / class_counts.min()

print("\n[2] Data Imbalance Check")
print(f"Imbalance Ratio (max/min): {imbalance_ratio:.2f}")

if imbalance_ratio <= 2:
    data_status = "SEIMBANG"
else:
    data_status = "IMBALANCE"

print(f"Status               : {data_status}")

# ------------------------------------------------------------------
# 3. METRIC CONSISTENCY CHECK
# ------------------------------------------------------------------
metric_gap = max(acc, prec, rec, f1) - min(acc, prec, rec, f1)

print("\n[3] Metric Consistency Check")
print(f"Accuracy  : {acc:.4f}")
print(f"Precision : {prec:.4f}")
print(f"Recall    : {rec:.4f}")
print(f"F1-score  : {f1:.4f}")
print(f"Metric Gap: {metric_gap:.4f}")

if metric_gap <= 0.02:
    metric_status = "KONSISTEN"
else:
    metric_status = "TIDAK KONSISTEN"

print(f"Status    : {metric_status}")

# ------------------------------------------------------------------
# 4. CONFIDENCE PREDICTION CHECK
# ------------------------------------------------------------------
confidences = np.max(pred_probs, axis=1)

mean_conf = confidences.mean() * 100

print("\n[4] Confidence Prediction Check")
print(f"Mean Confidence : {mean_conf:.2f}%")

if mean_conf >= 70:
    conf_status = "PREDIKSI MEYAKINKAN"
else:
    conf_status = "PREDIKSI KURANG MEYAKINKAN"

print(f"Status          : {conf_status}")

# ------------------------------------------------------------------
# 5. FINAL HEALTH VERDICT
# ------------------------------------------------------------------
print("\n===== FINAL MODEL HEALTH VERDICT =====")

if (
    acc_gap <= 0.05 and
    imbalance_ratio <= 2 and
    metric_gap <= 0.02 and
    mean_conf >= 70
):
    print("KESIMPULAN : ✅ MODEL SEHAT, STABIL, DAN LAYAK DIGUNAKAN")
else:
    print("KESIMPULAN : ⚠️ MODEL MEMERLUKAN OPTIMASI LANJUTAN")


In [ ]:
import json

# ==============================================================================
# 12. SAVE MODEL & LABEL MAPPING
# ==============================================================================
model.save("efficientnetb0_model.keras")

with open("class_indices.json", "w") as f:
    json.dump({v: k for k, v in train_ds.class_indices.items()}, f)

print("Model dan label mapping berhasil disimpan.")
